# CS 533 Assignment 1: College Scorecard

**by Arash Modaresi Rad**

## Project Requirements

Import libraries:

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

### 1. Load the most recent cohort’s data file.

According to CollegeScorecardDataDictionary.xlsx the most recent cohort's data file is MERGED2017_18_PP.csv

In [2]:
latest_cohort = pd.read_csv('MERGED2017_18_PP.csv', low_memory = False)

FileNotFoundError: [Errno 2] File b'MERGED2017_18_PP.csv' does not exist: b'MERGED2017_18_PP.csv'

### 2. Describe the data set’s basic characteristics:

In [ ]:
latest_cohort.info()

The 2017-18 data from college scorecards show that there are 7115 rows, and 1977 columns.
Also this dataset contains 1894 float64, 14 int64, and 69 object typs of data.

In [ ]:
latest_cohort.head()

#### a. How many schools are there?

All schools are identified with a unique identification number namely `UNITID`

In [ ]:
print('Number of schools with unique location: ', latest_cohort['UNITID'].nunique())

This is equall to the number of rows of the dataset which means all of the elements are unique schools with separate locations

But if we consider that several locations of an institution is considered one institution then `OPEID6` must be used to count the number of schools

In [ ]:
print('Number of schools considering several locations of an institution exists: ', latest_cohort['OPEID6'].nunique())

#### b. How many main and branch schools are there? (the variable MAIN is 1 for a main campus, and 0 for a branch)

In [ ]:
print('The number of main schools: ', Counter(latest_cohort['MAIN']).get(1))

In [ ]:
print('The number of btanch schools: ', Counter(latest_cohort['MAIN']).get(0))

#### c. How many schools are there per state?

Assumeing that school branches are considered as seperate schools we can count the number of elements in each state 

In [ ]:
AllSchools = Counter(latest_cohort['STABBR'])
Num_allSchools = pd.DataFrame(AllSchools, index = [0])
print('Number of all schools including the branches per state: ')
Num_allSchools


Assumeing that school branches are considered as one school we can count the number of elements in each state by removing the duplicated values in `OPEID6` and counting the numbers per state

In [ ]:
new_list1 = pd.DataFrame(latest_cohort);
droped_list = new_list1.drop_duplicates(subset = ['OPEID6'], keep = 'first')
## just for visual confirmation
#bbb = bb.sort_values('OPEID6', ascending = True) 
Num_schools2 = pd.DataFrame(Counter(droped_list['STABBR']), index = [0])
print('Number of all schools excluding the branches per state:')
Num_schools2


#### d. How are schools-per-state distributed? Describe this both numerically and with a plot.


Plotting the bar graphs of number of schools per state with two scenarios of 

considering school branches as seperate schools (`excluding branches`) and 

not considering school branches as seperate schools (`excluding branches`)

In [ ]:
ax = list(Num_allSchools.columns)
ax1 = np.transpose(ax)
#print(ax1)
ax2 = Num_schools2.values[0]
ax3 = Num_allSchools.values[0]
#print(ax2)
#print(np.arange(len(ax2)))
plt.bar(np.arange(len(ax2)), ax2, align = 'center', alpha = 1)
plt.bar(np.arange(len(ax3)), ax3, align = 'center', alpha = 0.5)
plt.legend(['Excluding branches', 'Including branches'], loc = 1)
plt.xticks(np.arange(len(ax2)), ax1, rotation = 'vertical')
plt.tick_params(labelsize = 5)
plt.ylabel('Number of schools')
plt.xlabel('States')
plt.title('Plot bar of schools per state')
plt.show()

In [ ]:
# convert lists to dictionary 
res = {'excluding branches': [], 'Including branches': []}
range_list = np.arange(len(ax2))
for i in range_list:
    res['excluding branches'].append(ax2[i]) 
    res['Including branches'].append(ax3[i])  

# calculate stats
stat1 = pd.DataFrame(res['excluding branches']).describe()
stat2 = pd.DataFrame(res['Including branches']).describe()
print('ststistical properties of schools-per-state (excluding branches case): \n', stat1)
print('\nststistical properties of schools-per-state (Including branches case): \n', stat2)

It is evident that in both cases the median is lower than the mean indicating positive skewness and in both cases the **median** is more appropriate to represent central tendency.

In [ ]:
plt.hist(ax2, density = True, bins = 10) 
plt.hist(ax3, density = True, bins = 10, alpha = 0.5) 
plt.legend(['Excluding branches', 'Including branches'], loc = 1)
plt.ylabel('Frequency');
plt.xlabel('Value');
plt.title('Histogram of number of schools per state')

### 3. Examine numeric statistics - what are their averages? Think about what average is most appropriate! How are they distributed?

#### a. School enrollment (size)

Enrollment of all undergraduate students is a varaible named `UGDS` and it is an integer

In [ ]:
School_enrollment = latest_cohort['UGDS']
stat_School_enrollment = pd.DataFrame(School_enrollment).describe()
print(stat_School_enrollment)

Plot the histogram

In [ ]:
plt.hist(School_enrollment, density = True, bins = 30) 
plt.ylabel('Frequency');
plt.xlabel('Value');
plt.title('Histogram of number of schools enrollment')
thirdQ = latest_cohort['UGDS'].describe().get('75%').astype('float64')
firstQ  = latest_cohort['UGDS'].describe().get('25%').astype('float64')
print('Inter-quartile range: ', np.subtract(thirdQ, firstQ))

It is evident that our data exhibits a positive skewness and therefore median is more appropriate to represent central tendency.
Also both standrad deviation and the inter-quartile range are large (5481 and 1912, respectively) indicating large variablility between diferent schools

Now considering that **all school branches are considered as one school** we have to first groupby `OPEID6` and then sum their values so that all branches represent one school and then calculate the stats of school enrollment size

In [ ]:
df2 = latest_cohort.groupby(['OPEID6'], as_index = False).sum()
stat_School_enrollment2 = df2['UGDS'].describe()
print(stat_School_enrollment2)
#print(stat_School_enrollment2.iloc[2])

Plot the histogram of school enrollment as **values of enrollment in all branches of the same school are sumed and represented as one school**

In [ ]:
plt.hist(df2['UGDS'], density = True, bins = 30) 
plt.ylabel('Frequency');
plt.xlabel('Value');
plt.title('Histogram of number of school enrollments')
thirdQ = stat_School_enrollment2.iloc[6].astype('float64')
firstQ  = stat_School_enrollment2.iloc[4].astype('float64')
print('Inter-quartile range: ', np.subtract(thirdQ, firstQ))

As before, the data exhibits a positive skewness and therefore median is more appropriate to represent central tendency.
Also both standrad deviation and the inter-quartile range are large (6145 and 2480, respectively) indicating large variablility between diferent schools.

#### b. Completion rate

#### c. Admission rate

To assess the admission rate of diffrent schools, the two varaibles of `ADM_RATE` and `ADM_RATE_ALL` will be used, where the first is admission rate of all schools and the seccond merges the school branches and represents the overall admission rate of the school by name

In [ ]:
Admission = latest_cohort['ADM_RATE']
stat_Admission = pd.DataFrame(Admission).describe()
print(stat_Admission)

In [ ]:
Admission_All = latest_cohort['ADM_RATE_ALL']
stat_Admission_All = pd.DataFrame(Admission_All).describe()
print(stat_Admission_All)

Plot the histogram of admission rare as **both total admission rate and local schools admission rates**

In [ ]:
plt.hist(Admission, density = True, bins = 30) 
plt.hist(Admission_All, density = True, bins = 30, alpha = 0.7) 
plt.ylabel('Frequency')
plt.xlabel('Value')
plt.title('Histogram of admission rate')
plt.legend(['Local admission rate', 'Total admission rate'], loc = 2)
thirdQ = stat_Admission.iloc[6].astype('float64')
firstQ  = stat_Admission.iloc[4].astype('float64')
thirdQ_All = stat_Admission_All.iloc[6].astype('float64')
firstQ_All = stat_Admission_All.iloc[4].astype('float64')
print('Admission rate Inter-quartile range: ', np.subtract(thirdQ, firstQ))
print('Total dmission rate Inter-quartile range: ', np.subtract(thirdQ_All, firstQ_All))

The admission data exhibits a negative skewness and the diffrence between mean and median is both cases is insignifcant and therefore, both are appropriate for representing centarl tendency of data.
Also both standrad deviation and the inter-quartile range are not very large (~0.2 and ~0.3, respectively) indicating medium variablility between diferent schools.

#### d. Tuition & fees

#### e. Cost (Average Cost of Attendance, COSTT4_A/COSTT4_P)

For this analysis the `COSTT4_A` data are used that represent the average annual total cost of attendance, including tuition and fees, books and supplies, and living expenses for all full-time, first-time, degree/certificate-seeking undergraduates who receive Title IV aid.

In [ ]:
Cost = latest_cohort['COSTT4_A']
stat_Cost = pd.DataFrame(Cost).describe()
print(stat_Cost)

Plot the histogram of average cost of attendance

In [ ]:
plt.hist(Cost, density = True, bins = 30) 
plt.ylabel('Frequency');
plt.xlabel('Value');
plt.title('Histogram of cost of attendance')
thirdQ = stat_Cost.iloc[6].astype('float64')
firstQ  = stat_Cost.iloc[4].astype('float64')
print('Inter-quartile range: ', np.subtract(thirdQ, firstQ))

The cost of attendance data exhibits a positive skewness and the diffrence between mean and median is both cases is signifcant fue to presence of few number of large data in our dataset and therefore, median is appropriate for representing centarl tendency of data.
Also both standrad deviation and the inter-quartile range are very large (15015 and 19941, respectively)indicating high variablility between diferent schools.

### 4. Break down completion rate by school characteristics (box plots are useful for this):